1\. Write a function that converts number representation (bin<->dec<->hex)

In [ ]:
def convert(x, in_base, out_base):
    if in_base == "dec":
        if out_base == "dec":
            return x
        elif out_base == "bin":
            return bin(x)
        elif out_base == "hex":
            return hex(x)
    if in_base == "bin":
        if out_base == "dec":
            return int(x, 2)
        elif out_base == "bin":
            return x
        elif out_base == "hex":
            return hex(int(x, 2))
    if in_base == "hex":
        if out_base == "dec":
            return int(x, 16)
        elif out_base == "bin":
            return bin(int(x, 16))
        elif out_base == "hex":
            return x
    
print(convert(18, "dec", "hex")) 
print(convert(18, "dec", "dec")) 
print(convert('0b1100', "bin", "dec")) 
print(convert('0b1100', "bin", "hex")) 
print(convert('0xCD00', "hex", "dec")) 

2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [ ]:
def to_float(bits):
    # sign
    sign = (-1)**(int(bits[0]))
    
    #exponent
    exponent = bits[1:9]
    exp = 0;
    k = len(exponent) - 1
    for b in exponent:
        exp += int(b)*(2**k)
        k -= 1
    exp -= 127
    
    #mantissa
    mantissa = bits[9:]
    frac = 0
    k = 1
    for b in mantissa:
        frac += int(b)*(2**(-k))
        k = k+1
    frac += 1
    
    dec = sign*(2**exp)*frac
    return dec

stringa = '11000001110011000000000100000000'
print(to_float(stringa))

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [ ]:
uflow = 1
ucount = 0
while uflow != 0:
    ucount += 1
    min_n = uflow
    uflow = uflow/2
print("Underflow limit reached after " + str(ucount) + " iterations, equal to " + str(min_n))


oflow = 1
ocount = 0
while oflow/oflow == 1: # NOT WORK
    ocount += 1
    max_n = oflow
    oflow = oflow*2
print("Overflow limit reached after " + str(ocount) + " iterations, equal to " + str(max_n))

4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [ ]:
n = 1
epsilon = 1
m = n + epsilon
while n != m:
    epsilon = epsilon/2
    m = n + epsilon

print("The two variable results the same after adding an epsilon equal to: " + str(epsilon))

5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [ ]:
import math as m
a = 0.001
b = 1000
c = 0.001

def compute_x(a, b, c):
    delta = (b**2) - (4*a*c)
    x1 = (-b + m.sqrt(delta)) / (2*a)
    x2 = (-b - m.sqrt(delta)) / (2*a)
    return x1, x2

def compute_x_mod(a, b, c):
    delta = (b**2) - (4*a*c)
    x1 =  (2*c) / (-b - m.sqrt(delta))
    x2 =  (2*c) / (-b + m.sqrt(delta))
    return x1, x2

print(compute_x(a, b, c))
print(compute_x_mod(a, b, c))

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [ ]:
def f(x):
    return x*(x-1)

def derivative(func , x, delta):
    return ((func(x+delta)-func(x))/delta)

print("Derivative of f(x=1) with delta =", pow(10, -2), "\t\t:", derivative(f, 1, pow(10, -2)))

deltas = [pow(10, -4), pow(10, -6), pow(10, -8), pow(10, -10), pow(10, -12), pow(10, -14)]
for i in deltas:
    print("Derivative of f(x=1) with delta =", i, "\t:", derivative(f, 1, i))

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [ ]:
import math as m

def func(x):
    return m.sqrt(1-(x**2))


def integral(start, end, function, N):
    h = end - start
    start = start*N
    end = end*N
    I = sum( [ (h/N)*function(x/N) for x in range(start,end,h) ] )
    return I

print("Value of the integral with N = 100 :", integral(-1,1,func, 100))

import time
for n in range(10**5,10**10,10**5):
    t0 = time.process_time()
    I = integral(-1,1, func, n)
    t1 = time.process_time()
    if t1-t0 < 1:    
        print("N =", n , "\tTime =", (t1-t0), "s", "\tResult =", I)
    else:
        break
print("\nTo calculate the integral in lower than a second, the maximum value of N is about:", n-10**5)
print("The difference between the true value is:", abs(integral(-1,1, func, n-(10**5)) - ((m.pi)/2)))

In [ ]:
# Since expended time doubles when the number of iterations is doubled
# If we multiply by 60 the maximum number N we should reach an
# execution time of about one minute

N = (n-10**5)*60
t0 = time.process_time()
I = integral(-1,1, func, N)
t1 = time.process_time()
print("N =", N , "\tTime =", (t1-t0), "s", "\tResult =", I)
print("The difference with respect to the true value is:", abs(I - ((m.pi)/2)))
print("The difference with respect to the previous error is:", abs(I - integral(-1,1, func, n-(10**5)))